In [1]:
# This is my main readiness code. 
# Prompts you to select folder and adds info into sqlite database at Readiness_Screen_Data.db

import os
import sqlite3
import tkinter as tk
from tkinter import filedialog
import xml.etree.ElementTree as ET
import pandas as pd

# Database file path
db_path = 'D:/Readiness Screen 3/Readiness_Screen_Data_v2.db'

# Establish connection and create tables
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# -------------------------------------------------
# 1. Create/Update Tables with Correct Column Order
# -------------------------------------------------
cursor.executescript("""
CREATE TABLE IF NOT EXISTS Participant (
    Participant_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Height REAL,
    Weight REAL,
    Plyo_Day TEXT,
    Creation_Date TEXT
);

CREATE TABLE IF NOT EXISTS I (
    Trial_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Participant_ID INTEGER,
    Avg_Force REAL,
    Avg_Force_Norm REAL,
    Max_Force REAL,
    Max_Force_Norm REAL,
    Time_to_Max REAL,
    Creation_Date TEXT,
    FOREIGN KEY (Participant_ID) REFERENCES Participant(Participant_ID)
);

CREATE TABLE IF NOT EXISTS Y (
    Trial_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Participant_ID INTEGER,
    Avg_Force REAL,
    Avg_Force_Norm REAL,
    Max_Force REAL,
    Max_Force_Norm REAL,
    Time_to_Max REAL,
    Creation_Date TEXT,
    FOREIGN KEY (Participant_ID) REFERENCES Participant(Participant_ID)
);

CREATE TABLE IF NOT EXISTS T (
    Trial_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Participant_ID INTEGER,
    Avg_Force REAL,
    Avg_Force_Norm REAL,
    Max_Force REAL,
    Max_Force_Norm REAL,
    Time_to_Max REAL,
    Creation_Date TEXT,
    FOREIGN KEY (Participant_ID) REFERENCES Participant(Participant_ID)
);

CREATE TABLE IF NOT EXISTS IR90 (
    Trial_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Participant_ID INTEGER,
    Avg_Force REAL,
    Avg_Force_Norm REAL,
    Max_Force REAL,
    Max_Force_Norm REAL,
    Time_to_Max REAL,
    Creation_Date TEXT,
    FOREIGN KEY (Participant_ID) REFERENCES Participant(Participant_ID)
);

CREATE TABLE IF NOT EXISTS CMJ (
    Trial_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Participant_ID INTEGER,
    Jump_Height REAL,
    Peak_Power REAL,
    Peak_Force REAL,
    Creation_Date TEXT,
    FOREIGN KEY (Participant_ID) REFERENCES Participant(Participant_ID)
);

CREATE TABLE IF NOT EXISTS PPU (
    Trial_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name TEXT,
    Participant_ID INTEGER,
    Jump_Height REAL,
    Peak_Power REAL,
    Peak_Force REAL,
    Creation_Date TEXT,
    FOREIGN KEY (Participant_ID) REFERENCES Participant(Participant_ID)
);
""")
conn.commit()

# ---------- add new columns to CMJ & PPU if they aren’t already present -------
for tbl in ("CMJ", "PPU"):
    cursor.execute(f"PRAGMA table_info({tbl});")
    cols = {row[1] for row in cursor.fetchall()}
    for col_sql in (
        "ADD COLUMN Jump_Height        REAL",
        "ADD COLUMN Peak_Power         REAL",
        "ADD COLUMN Peak_Force         REAL",
        "ADD COLUMN PP_W_per_kg        REAL",
        "ADD COLUMN PP_FORCEPLATE      REAL",
        "ADD COLUMN Force_at_PP        REAL",
        "ADD COLUMN Vel_at_PP          REAL",
        "ADD COLUMN Creation_Date      TEXT"
    ):
        col_name = col_sql.split()[2]
        if col_name not in cols:
            cursor.execute(f"ALTER TABLE {tbl} {col_sql}")

conn.commit()

# -------------------------------------------------
# 2. Prompt user to select a folder
# -------------------------------------------------
root = tk.Tk()
root.withdraw()
selected_folder = filedialog.askdirectory(initialdir='D:/Readiness Screen 3/Data/')

if not selected_folder:
    print("No folder selected. Exiting...")
    exit()

# -------------------------------------------------
# 3. Locate the XML file (assuming 'sessionXYZ.xml')
# -------------------------------------------------
xml_file_path = ''
for root_dir, _, files in os.walk(selected_folder):
    for file in files:
        if file.lower().startswith('session') and file.lower().endswith('.xml'):
            xml_file_path = os.path.join(root_dir, file)
            break
    if xml_file_path:
        break

if not xml_file_path:
    print("No XML file found. Exiting...")
    exit()

# -------------------------------------------------
# 4. Parse the XML file
# -------------------------------------------------
tree = ET.parse(xml_file_path)
xml_root = tree.getroot()

def find_text(element, tag):
    found = element.find(tag)
    return found.text if found is not None else None

session_fields = xml_root.find(".//Session/Fields")
name = find_text(session_fields, "Name")
height = find_text(session_fields, "Height")
weight = find_text(session_fields, "Weight")
plyo_day = find_text(session_fields, "Plyo_Day")
creation_date = find_text(session_fields, "Creation_date")

if None in [name, height, weight, plyo_day, creation_date]:
    print("Missing data in XML file. Exiting...")
    exit()

# -------------------------------------------------
# 5. Insert participant data
# -------------------------------------------------
cursor.execute("""
INSERT INTO Participant (Name, Height, Weight, Plyo_Day, Creation_Date)
VALUES (?, ?, ?, ?, ?)
""", (name, height, weight, plyo_day, creation_date))

participant_id = cursor.lastrowid
conn.commit()

# -------------------------------------------------
# 6. Define ASCII file mapping and output path
# -------------------------------------------------
ascii_files = {
    "I"  :"i_data.txt",   "Y"  :"y_data.txt",
    "T"  :"t_data.txt",   "IR90":"ir90_data.txt",
    "CMJ":"cmj_data.txt", "PPU":"ppu_data.txt"
}
output_path = r'D:/Readiness Screen 3/Output Files/'

for key, fname in ascii_files.items():
    fpath = os.path.join(output_path, fname)
    if not os.path.exists(fpath):
        print(f"(skip) {fname} not found"); continue

    if key in {"CMJ","PPU"}:
        headers = ["JH_IN", "LEWIS_PEAK_POWER", "Max_Force",
                   "PP_W_per_kg", "PP_FORCEPLATE", "Force_at_PP", "Vel_at_PP"]  ### NEW ▶
    else:
        headers = ["Max_Force", "Max_Force_Norm",
                   "Avg_Force", "Avg_Force_Norm", "Time_to_Max"]

    df = pd.read_csv(fpath, sep=r'\s+', skiprows=5, names=headers)
    print(fname, "preview\n", df.head())

    for _, row in df.iterrows():
        if key in {"CMJ","PPU"}:
            cursor.execute(f"""
                INSERT INTO {key}(
                    Name, Participant_ID,
                    Jump_Height,        -- JH_IN
                    Peak_Power,         -- legacy Lewis
                    Peak_Force,         -- Max_Force
                    PP_W_per_kg, PP_FORCEPLATE,
                    Force_at_PP, Vel_at_PP,
                    Creation_Date
                ) VALUES (?,?,?,?,?,?,?,?,?,?)
            """, (
                name, participant_id,
                row['JH_IN'], row['LEWIS_PEAK_POWER'], row['Max_Force'],
                row['PP_W_per_kg'], row['PP_FORCEPLATE'],
                row['Force_at_PP'],  row['Vel_at_PP'],
                creation_date
            ))
        else:
            # I / Y / T / IR90 (unchanged)
            cursor.execute(f"""
              INSERT INTO {key}(
                    Name,Participant_ID,Avg_Force,Avg_Force_Norm,
                    Max_Force,Max_Force_Norm,Time_to_Max,Creation_Date)
              VALUES (?,?,?,?,?,?,?,?)
            """,(
                name,participant_id,
                row['Avg_Force'],row['Avg_Force_Norm'],
                row['Max_Force'],row['Max_Force_Norm'],
                row['Time_to_Max'],creation_date
            ))

# -------------------------------------------------
# 8. Final Commit and Close
# -------------------------------------------------
conn.commit()
conn.close()

print("Data successfully added to the database.")


i_data.txt preview
    Max_Force  Max_Force_Norm  Avg_Force  Avg_Force_Norm  Time_to_Max
1      143.2             1.9      111.9            1.51         3.36
y_data.txt preview
    Max_Force  Max_Force_Norm  Avg_Force  Avg_Force_Norm  Time_to_Max
1      119.8             1.6       97.1            1.31         1.58
t_data.txt preview
    Max_Force  Max_Force_Norm  Avg_Force  Avg_Force_Norm  Time_to_Max
1      108.7             1.5       93.3            1.26         1.02
ir90_data.txt preview
    Max_Force  Max_Force_Norm  Avg_Force  Avg_Force_Norm  Time_to_Max
1       99.6             1.3       88.9             1.2         2.73
cmj_data.txt preview
    JH_IN  LEWIS_PEAK_POWER  Max_Force  PP_W_per_kg  PP_FORCEPLATE  \
1  18.96           7464.55    1847.52         6.57         485.51   

   Force_at_PP  Vel_at_PP  
1      1771.21     274.11  
ppu_data.txt preview
    JH_IN  LEWIS_PEAK_POWER  Max_Force  PP_W_per_kg  PP_FORCEPLATE  \
1   6.26           5467.41    1020.52         2.97       

In [2]:
# Reorders the database to be in alphabetical order

import sqlite3


db_path = "D:/Readiness Screen 3/Readiness_Screen_Data_v2.db" 
sort_column = "Name"     

def reorder_all_tables(db_path, sort_column):
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Fetch all table names in the database
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        for table in tables:
            table_name = table[0]

            # Skip system tables like sqlite_sequence
            if table_name.startswith("sqlite_"):
                continue

            print(f"Processing table: {table_name}")

            # Check if the column exists in the current table
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = [info[1] for info in cursor.fetchall()]
            if sort_column not in columns:
                print(f"Skipping table '{table_name}' - Column '{sort_column}' not found.")
                continue

            # Create a new sorted table
            temp_table = f"{table_name}_sorted"
            cursor.execute(f"CREATE TABLE {temp_table} AS SELECT * FROM {table_name} ORDER BY {sort_column} ASC;")
            
            # Drop the old table
            cursor.execute(f"DROP TABLE {table_name};")
            
            # Rename the new table to the original name
            cursor.execute(f"ALTER TABLE {temp_table} RENAME TO {table_name};")
            print(f"Table '{table_name}' reordered successfully.")

        # Commit changes
        conn.commit()
        print("All tables processed.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()

reorder_all_tables(db_path, sort_column)


Processing table: Participant
Table 'Participant' reordered successfully.
Processing table: I
Table 'I' reordered successfully.
Processing table: Y
Table 'Y' reordered successfully.
Processing table: T
Table 'T' reordered successfully.
Processing table: IR90
Table 'IR90' reordered successfully.
Processing table: CMJ
Table 'CMJ' reordered successfully.
Processing table: PPU
Table 'PPU' reordered successfully.
All tables processed.


In [3]:
# ──────────────────────────────────────────────────────────────────────────────
#  DASH REPORT  –  Readiness Dashboard v3
# ──────────────────────────────────────────────────────────────────────────────
import sqlite3, pandas as pd, numpy as np
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.colors as plotly_colors

db = r'D:/Readiness Screen 3/Readiness_Screen_Data_v2.db'
conn = sqlite3.connect(db)

# ─── pull data ───────────────────────────────────────────────────────────────
df_cmj = pd.read_sql("""
    SELECT Name, Creation_Date,
           Jump_Height             AS Jump_Height_CMJ,
           PP_FORCEPLATE           AS PP_FORCEPLATE_CMJ,
           Force_at_PP             AS Force_at_PP_CMJ,
           Vel_at_PP               AS Vel_at_PP_CMJ
    FROM CMJ
""", conn)

df_ppu = pd.read_sql("""
    SELECT Name, Creation_Date,
           Jump_Height             AS Jump_Height_PPU,
           PP_FORCEPLATE           AS PP_FORCEPLATE_PPU,
           Force_at_PP             AS Force_at_PP_PPU,
           Vel_at_PP               AS Vel_at_PP_PPU
    FROM PPU
""", conn)

df_i   = pd.read_sql("SELECT Name, Creation_Date, Avg_Force AS Avg_Force_I   FROM I"   , conn)
df_y   = pd.read_sql("SELECT Name, Creation_Date, Avg_Force AS Avg_Force_Y   FROM Y"   , conn)
df_t   = pd.read_sql("SELECT Name, Creation_Date, Avg_Force AS Avg_Force_T   FROM T"   , conn)
df_ir  = pd.read_sql("SELECT Name, Creation_Date, Avg_Force AS Avg_Force_IR90 FROM IR90", conn)
conn.close()

# merged for the time-series rows
df_merged = (
    df_cmj.merge(df_ppu, on=["Name", "Creation_Date"], how="outer")
          .merge(df_i , on=["Name", "Creation_Date"], how="outer")
          .merge(df_y , on=["Name", "Creation_Date"], how="outer")
          .merge(df_t , on=["Name", "Creation_Date"], how="outer")
          .merge(df_ir, on=["Name", "Creation_Date"], how="outer")
)
df_merged["Creation_Date"] = pd.to_datetime(df_merged["Creation_Date"])
df_merged.sort_values("Creation_Date", inplace=True)

participants = sorted(df_merged["Name"].dropna().unique())          ### NEW ▶

# Create reference dataframes with proper filtering
cmj_ref = df_cmj[(df_cmj["Force_at_PP_CMJ"].notna()) & (df_cmj["Vel_at_PP_CMJ"].notna())].copy()
ppu_ref = df_ppu[(df_ppu["Force_at_PP_PPU"].notna()) & (df_ppu["Vel_at_PP_PPU"].notna())].copy()

# ─── Dash app layout ─────────────────────────────────────────────────────────
app = Dash(__name__, title="Readiness Dashboard")

# Add custom CSS for dark dropdown
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>{%title%}</title>
        {%favicon%}
        {%css%}
        <style>
            .dark-dropdown .Select-control {
                background-color: #2d2d2d !important;
                border: 1px solid #444 !important;
                color: white !important;
            }
            .dark-dropdown .Select-value-label {
                color: white !important;
            }
            .dark-dropdown .Select-input {
                color: white !important;
            }
            .dark-dropdown .Select-menu-outer {
                background-color: #2d2d2d !important;
                border: 1px solid #444 !important;
            }
            .dark-dropdown .Select-option {
                background-color: #2d2d2d !important;
                color: white !important;
            }
            .dark-dropdown .Select-option.is-focused {
                background-color: #444 !important;
            }
            .dark-dropdown .Select-option.is-selected {
                background-color: #555 !important;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

def stat_box(lines):
    """helper that wraps a monospaced <pre>"""
    return html.Pre(lines, style={
        "background":"#2d2d2d","color":"#ffffff","padding":"30px",
        "borderRadius":"8px","fontFamily":"monospace","whiteSpace":"pre",
        "fontSize":"18px","border":"1px solid #444","margin":"10px 0",
        "boxShadow":"0 2px 4px rgba(0,0,0,0.3)","width":"100%",
        "textAlign":"center","letterSpacing":"1px"
    })

app.layout = html.Div(
    [
        html.H2("Readiness / Force-Plate Dashboard", style={"color":"white", "textAlign":"center", "marginBottom":"30px"}),

        # athlete picker
        html.Div([
            html.Label("Select athlete:", style={"color":"white", "fontSize":"16px", "fontWeight":"bold"}),
            dcc.Dropdown(
                id="athlete",
                options=[{"label":n, "value":n} for n in participants],
                value=participants[0] if participants else None,
                clearable=False,
                style={
                    "backgroundColor":"#2d2d2d", 
                    "color":"white",
                    "border":"1px solid #444"
                },
                className="dark-dropdown"
            ),
        ], style={"width":"400px", "marginBottom":"20px"}),

        html.Hr(style={"borderColor":"#444", "margin":"30px 0"}),

        # ───────── Row 1 – Avg-Force over time ─────────
        html.Div([
            dcc.Graph(id="force-lines", style={"width":"100%", "height":"600px", "marginBottom":"20px"}),
            html.Div(id="force-box", style={"width":"100%", "marginBottom":"30px"}),
        ]),

        html.Hr(style={"borderColor":"#444", "margin":"30px 0"}),

        # ───────── Row 2 – CMJ line + scatter ────
        html.Div([
            html.Div([
                dcc.Graph(id="cmj-jump", style={"width":"48%", "display":"inline-block", "height":"600px"}),
                dcc.Graph(id="cmj-scatter", style={"width":"48%", "display":"inline-block", "height":"600px", "marginLeft":"4%"}),
            ]),
            html.Div(id="cmj-box", style={"width":"100%", "marginTop":"20px", "marginBottom":"30px"}),
        ]),

        html.Hr(style={"borderColor":"#444", "margin":"30px 0"}),

        # ───────── Row 3 – PPU line + scatter ────
        html.Div([
            html.Div([
                dcc.Graph(id="ppu-jump", style={"width":"48%", "display":"inline-block", "height":"600px"}),
                dcc.Graph(id="ppu-scatter", style={"width":"48%", "display":"inline-block", "height":"600px", "marginLeft":"4%"}),
            ]),
            html.Div(id="ppu-box", style={"width":"100%", "marginTop":"20px", "marginBottom":"30px"}),
        ]),
    ],
    style={"maxWidth":"2200px","margin":"auto", "backgroundColor":"#1a1a1a", "color":"white", "minHeight":"100vh"},
)

# ─── utilities ───────────────────────────────────────────────────────────────
def last_two(df, col):
    df = df.dropna(subset=[col]).sort_values("Creation_Date")
    if len(df) < 2: return "–","–","–"
    latest, prev = df.iloc[-1][col], df.iloc[-2][col]
    return f"{latest:.2f}", f"{prev:.2f}", f"{latest-prev:+.2f}"

# ─── callback ────────────────────────────────────────────────────────────────
@app.callback(
    Output("force-lines" , "figure"),
    Output("cmj-jump"    , "figure"),
    Output("ppu-jump"    , "figure"),
    Output("cmj-scatter" , "figure"),
    Output("ppu-scatter" , "figure"),
    Output("force-box"   , "children"),
    Output("cmj-box"     , "children"),
    Output("ppu-box"     , "children"),
    Input("athlete","value"),
)
def update(name):
    # Recreate reference dataframes to ensure they have all columns
    global cmj_ref, ppu_ref
    cmj_ref = df_cmj[(df_cmj["Force_at_PP_CMJ"].notna()) & (df_cmj["Vel_at_PP_CMJ"].notna())].copy()
    ppu_ref = df_ppu[(df_ppu["Force_at_PP_PPU"].notna()) & (df_ppu["Vel_at_PP_PPU"].notna())].copy()
    
    dff = df_merged[df_merged["Name"] == name].sort_values("Creation_Date")
    dates_cat = dff["Creation_Date"].dt.strftime("%Y-%m-%d")        # category axis

    # ── Row 1: Avg-Force lines ──────────────────────────────────
    fig_force = go.Figure()
    for col,label in [("Avg_Force_I","I"),("Avg_Force_T","T"),
                      ("Avg_Force_Y","Y"),("Avg_Force_IR90","IR90")]:
        if dff[col].notna().any():
            fig_force.add_trace(go.Scatter(
                x=dates_cat,y=dff[col],mode="lines+markers",name=label))
    fig_force.update_layout(
        title="Avg Force (I / T / Y / IR90) – categorical spacing",
        template="plotly_dark",xaxis=dict(type="category"),
        xaxis_title="Session date",yaxis_title="Avg Force (N)",height=550)

    # stats for force box
    force_lines = ["Metric           Latest      Prev        Δ",
                   "──────────────── ────────── ────────── ──────────"]
    for col,label in [("Avg_Force_I","I"),("Avg_Force_T","T"),
                      ("Avg_Force_Y","Y"),("Avg_Force_IR90","IR90")]:
        l,p,d = last_two(dff,col)
        force_lines.append(f"{label:<15} {l:>10} {p:>10} {d:>10}")
    force_box = stat_box("\n".join(force_lines))

    # ── Row 2: CMJ jump-height line ─────────────────────────────
    fig_cmj = go.Figure()
    if dff["Jump_Height_CMJ"].notna().any():
        fig_cmj.add_trace(go.Scatter(
            x=dates_cat,y=dff["Jump_Height_CMJ"],
            mode="lines+markers",name="CMJ Jump Height"))
    fig_cmj.update_layout(
        title="CMJ Jump Height",
        template="plotly_dark",xaxis=dict(type="category"),
        xaxis_title="Session date",yaxis_title="JH (cm)",height=550)

    # CMJ scatter (Force x Vel with Vel on Y!)               ### NEW ▶ axis swap
    fig_c_scatter = go.Figure()
    fig_c_scatter.add_trace(go.Scatter(
        x=cmj_ref["Force_at_PP_CMJ"], y=cmj_ref["Vel_at_PP_CMJ"],
        mode="markers",name="Reference",
        marker=dict(color="cornflowerblue",opacity=0.4,size=8)))
    
    # Add selected athlete's data with different colors for each date
    sel = cmj_ref[cmj_ref["Name"]==name]
    if not sel.empty:
        # Get unique dates for this athlete
        unique_dates = sel["Creation_Date"].unique()
        color_palette = plotly_colors.qualitative.Set3  # Use a color palette with many colors
        
        for i, date in enumerate(unique_dates):
            date_data = sel[sel["Creation_Date"] == date]
            color = color_palette[i % len(color_palette)]
            
            fig_c_scatter.add_trace(go.Scatter(
                x=date_data["Force_at_PP_CMJ"], 
                y=date_data["Vel_at_PP_CMJ"],
                mode="markers+text", 
                textposition="top center",
                name=f"{name} ({date})", 
                marker=dict(color=color, size=12,
                line=dict(width=1, color="black")),
                hovertemplate=f"<b>{name}</b><br>Date: {date}<br>Force: %{{x}}<br>Velocity: %{{y}}<extra></extra>"
            ))
    fig_c_scatter.update_layout(
        title="CMJ Force-vs-Velocity",
        xaxis_title="Force @ PP (N)",yaxis_title="Velocity @ PP (m/s)",
        template="plotly_dark",height=550)

    cmj_lines = ["Metric                Latest      Prev        Δ",
                 "───────────────────── ────────── ────────── ──────────"]
    for col,label in [
        ("Jump_Height_CMJ"   ,"JH"),
        ("PP_FORCEPLATE_CMJ" ,"PP_FP"),
        ("Force_at_PP_CMJ"   ,"F@PP"),
        ("Vel_at_PP_CMJ"     ,"V@PP")]:
        l,p,d = last_two(dff,col); cmj_lines.append(f"{label:<20} {l:>10} {p:>10} {d:>10}")
    cmj_box = stat_box("\n".join(cmj_lines))

    # ── Row 3: PPU jump-height line ────────────────────────────
    fig_ppu = go.Figure()
    if dff["Jump_Height_PPU"].notna().any():
        fig_ppu.add_trace(go.Scatter(
            x=dates_cat,y=dff["Jump_Height_PPU"],
            mode="lines+markers",name="PPU Jump Height"))
    fig_ppu.update_layout(
        title="PPU Jump Height",
        template="plotly_dark",xaxis=dict(type="category"),
        xaxis_title="Session date",yaxis_title="JH (cm)",height=550)

    # PPU scatter (Vel @ PP on Y)                             ### NEW ▶ axis swap
    fig_p_scatter = go.Figure()
    fig_p_scatter.add_trace(go.Scatter(
        x=ppu_ref["Force_at_PP_PPU"], y=ppu_ref["Vel_at_PP_PPU"],
        mode="markers",name="Reference",
        marker=dict(color="cornflowerblue",opacity=0.4,size=8)))
    
    # Add selected athlete's data with different colors for each date
    sel = ppu_ref[ppu_ref["Name"]==name]
    if not sel.empty:
        # Get unique dates for this athlete
        unique_dates = sel["Creation_Date"].unique()
        color_palette = plotly_colors.qualitative.Set3  # Use a color palette with many colors
        
        for i, date in enumerate(unique_dates):
            date_data = sel[sel["Creation_Date"] == date]
            color = color_palette[i % len(color_palette)]
            
            fig_p_scatter.add_trace(go.Scatter(
                x=date_data["Force_at_PP_PPU"], 
                y=date_data["Vel_at_PP_PPU"],
                mode="markers+text", 
                textposition="top center",
                name=f"{name} ({date})", 
                marker=dict(color=color, size=12,
                line=dict(width=1, color="black")),
                hovertemplate=f"<b>{name}</b><br>Date: {date}<br>Force: %{{x}}<br>Velocity: %{{y}}<extra></extra>"
            ))
    fig_p_scatter.update_layout(
        title="PPU Force-vs-Velocity",
        xaxis_title="Force @ PP (N)",yaxis_title="Velocity @ PP (m/s)",
        template="plotly_dark",height=550)

    ppu_lines = ["Metric                Latest      Prev        Δ",
                 "───────────────────── ────────── ────────── ──────────"]
    for col,label in [
        ("Jump_Height_PPU"   ,"JH"),
        ("PP_FORCEPLATE_PPU" ,"PP_FP"),
        ("Force_at_PP_PPU"   ,"F@PP"),
        ("Vel_at_PP_PPU"     ,"V@PP")]:
        l,p,d = last_two(dff,col); ppu_lines.append(f"{label:<20} {l:>10} {p:>10} {d:>10}")
    ppu_box = stat_box("\n".join(ppu_lines))

    return (fig_force, fig_cmj, fig_ppu,
            fig_c_scatter, fig_p_scatter,
            force_box, cmj_box, ppu_box)

# ─── run server ──────────────────────────────────────────────────────────────
if __name__ == "__main__":
    app.run_server(port=8051, debug=True, use_reloader=False)


In [4]:
# ─────────────────────────────────────────────────────────────────────────────
#  CELL 4 – Generate PDF Readiness Report
# ─────────────────────────────────────────────────────────────────────────────
import os, tempfile, sqlite3, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.platypus import (SimpleDocTemplate, Paragraph, Spacer, Image,
                                Table, TableStyle, PageBreak)
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

db_path   = r"D:/Readiness Screen 3/Readiness_Screen_Data_v2.db"
out_dir   = r"D:/Readiness Screen 3/Reports"
os.makedirs(out_dir, exist_ok=True)

# ------------------------------------------------------------------- helpers
def percentile(arr, value):
    """return percentile rank (0-100) of value within arr"""
    return round(100 * (np.sum(arr < value) / len(arr)), 1)

def hist_with_marker(ref, x, title, fname, xlabel):
    plt.figure(figsize=(6,4), facecolor="#333333")          # grey frame
    ax = plt.gca()
    ax.set_facecolor("#333333")
    ax.hist(ref, bins=25, color="cornflowerblue", alpha=.8, edgecolor="black")
    ax.axvline(x, color="red", linewidth=2)
    ax.set_xlabel(xlabel, color="white")
    ax.set_ylabel("Frequency", color="white")
    ax.set_title(title, color="white")
    ax.grid(color="black", linewidth=.4, alpha=.6)          # black grid
    ax.tick_params(colors="white")
    plt.tight_layout()
    plt.savefig(fname, facecolor="#333333")
    plt.close()


def scatter_ref(ref_x, ref_y, x, y, title, fname, xlabel, ylabel):
    plt.figure(figsize=(6,4), facecolor="#333333")
    ax = plt.gca(); ax.set_facecolor("#333333")
    ax.scatter(ref_x, ref_y, color="cornflowerblue", alpha=.5, edgecolor="none")
    ax.scatter(x, y, color="red", edgecolor="black", s=120)
    ax.set_xlabel(xlabel, color="white")
    ax.set_ylabel(ylabel, color="white")
    ax.set_title(title,  color="white")
    ax.grid(color="black", linewidth=.4, alpha=.6)
    ax.tick_params(colors="white")
    plt.tight_layout()
    plt.savefig(fname, facecolor="#333333")
    plt.close()

# ---------------------------------------------------------------- data pull
with sqlite3.connect(db_path) as conn:
    # 1️⃣ find the latest session **that was just inserted** for this athlete
    latest = pd.read_sql("""SELECT MAX(Creation_Date) AS d
                            FROM CMJ WHERE Name = ?""", conn, params=(name,))
    session_date = latest.loc[0, "d"] or creation_date

    # ---- Avg-Force tables ---------------------------------------------------
    forces = {}
    ref_forces = {}
    for tbl, label in [("I","I"),("T","T"),("Y","Y"),("IR90","IR90")]:
        q = f"SELECT Avg_Force FROM {tbl} WHERE Name=? AND Creation_Date=?"
        forces[label] = pd.read_sql(q, conn, params=(name, session_date)
                                    )["Avg_Force"].iat[0]
        ref_forces[label] = pd.read_sql(f"SELECT Avg_Force FROM {tbl}", conn
                                        )["Avg_Force"].dropna().values

    # ---- CMJ / PPU tables ---------------------------------------------------
    cmj   = pd.read_sql(
        """SELECT * FROM CMJ WHERE Name=? AND Creation_Date=?""", conn,
        params=(name, session_date)).iloc[0]
    ppu   = pd.read_sql(
        """SELECT * FROM PPU WHERE Name=? AND Creation_Date=?""", conn,
        params=(name, session_date)).iloc[0]
    cmj_ref = pd.read_sql("SELECT * FROM CMJ", conn)
    ppu_ref = pd.read_sql("SELECT * FROM PPU", conn)

# ---------------------------------------------------------------- graphics
tmpdir = tempfile.mkdtemp()

# Avg-Force hists
force_imgs = []
for lbl in forces:
    fn = os.path.join(tmpdir, f"{lbl}_hist.png")
    hist_with_marker(ref_forces[lbl], forces[lbl],
                     f"{lbl} Avg Force", fn, "Avg Force (N)")
    force_imgs.append(fn)

# ─── CMJ & PPU  (aliased)  – keep INSIDE the with-block ──────────────────────
    cmj = pd.read_sql(
        """
        SELECT Jump_Height   AS Jump_Height_CMJ,
               PP_FORCEPLATE AS PP_FORCEPLATE_CMJ,
               Force_at_PP   AS Force_at_PP_CMJ,
               Vel_at_PP     AS Vel_at_PP_CMJ
        FROM CMJ
        WHERE Name = ? AND Creation_Date = ?
        """,
        conn, params=(name, session_date)
    ).iloc[0]

    ppu = pd.read_sql(
        """
        SELECT Jump_Height   AS Jump_Height_PPU,
               PP_FORCEPLATE AS PP_FORCEPLATE_PPU,
               Force_at_PP   AS Force_at_PP_PPU,
               Vel_at_PP     AS Vel_at_PP_PPU
        FROM PPU
        WHERE Name = ? AND Creation_Date = ?
        """,
        conn, params=(name, session_date)
    ).iloc[0]

    cmj_ref = pd.read_sql(
        """
        SELECT Jump_Height   AS Jump_Height_CMJ,
               PP_FORCEPLATE AS PP_FORCEPLATE_CMJ,
               Force_at_PP   AS Force_at_PP_CMJ,
               Vel_at_PP     AS Vel_at_PP_CMJ
        FROM CMJ
        """, conn)

    ppu_ref = pd.read_sql(
        """
        SELECT Jump_Height   AS Jump_Height_PPU,
               PP_FORCEPLATE AS PP_FORCEPLATE_PPU,
               Force_at_PP   AS Force_at_PP_PPU,
               Vel_at_PP     AS Vel_at_PP_PPU
        FROM PPU
        """, conn)
# ─────────────────────────────────────────────────────────────────────────────


# ─── graphics & PDF (run AFTER the with-block) ───────────────────────────────
tmpdir = tempfile.mkdtemp()

def save_rgb(path):
    from PIL import Image
    Image.open(path).convert("RGB").save(path, format="PNG")

# CMJ / PPU histograms & scatters
cmj_hist = os.path.join(tmpdir, "cmj_hist.png")
hist_with_marker(cmj_ref["Jump_Height_CMJ"], cmj["Jump_Height_CMJ"],
                 "CMJ Jump Height", cmj_hist, "Jump Height (cm)")

cmj_scat = os.path.join(tmpdir, "cmj_scatter.png")
scatter_ref(cmj_ref["Force_at_PP_CMJ"], cmj_ref["Vel_at_PP_CMJ"],
            cmj["Force_at_PP_CMJ"], cmj["Vel_at_PP_CMJ"],
            "CMJ Force vs Velocity", cmj_scat,
            "Force @ PP (N)", "Velocity @ PP (m/s)")

ppu_hist = os.path.join(tmpdir, "ppu_hist.png")
hist_with_marker(ppu_ref["Jump_Height_PPU"], ppu["Jump_Height_PPU"],
                 "PPU Jump Height", ppu_hist, "Jump Height (cm)")

ppu_scat = os.path.join(tmpdir, "ppu_scatter.png")
scatter_ref(ppu_ref["Force_at_PP_PPU"], ppu_ref["Vel_at_PP_PPU"],
            ppu["Force_at_PP_PPU"], ppu["Vel_at_PP_PPU"],
            "PPU Force vs Velocity", ppu_scat,
            "Force @ PP (N)", "Velocity @ PP (m/s)")

cmj_pp_hist = os.path.join(tmpdir, "cmj_pp_hist.png")
hist_with_marker(cmj_ref["PP_FORCEPLATE_CMJ"],
                 cmj["PP_FORCEPLATE_CMJ"],
                 "CMJ Peak Power", cmj_pp_hist, "PP ForcePlate (W)")

ppu_pp_hist = os.path.join(tmpdir, "ppu_pp_hist.png")
hist_with_marker(ppu_ref["PP_FORCEPLATE_PPU"],
                 ppu["PP_FORCEPLATE_PPU"],
                 "PPU Peak Power", ppu_pp_hist, "PP ForcePlate (W)")
# convert to 24-bit RGB for ReportLab
for p in [cmj_hist, cmj_pp_hist, cmj_scat,
          ppu_hist, ppu_pp_hist, ppu_scat] + force_imgs:
    save_rgb(p)

# ─────────────────────────────────────────────────────────────────────────────
#  PDF LAYOUT – CMJ row, PPU row, then force-hist row
# ─────────────────────────────────────────────────────────────────────────────
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.units import mm
from reportlab.platypus import Table, TableStyle, Paragraph
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

# colours
BRAND_BORDER = colors.HexColor("#4887a8")
CARD_BG      = colors.HexColor("#1f1f1f")
TEXT_COLOR   = colors.white
LOGO_PATH    = r"C:\Users\q\PycharmProjects\Readiness Screen\8ctane - Faded 8 to Blue.png"
# ── PAGE SIZE  (landscape A4 width, extra height) ────────────────────────────
# ── PAGE- & CARD-LEVEL CONSTANTS ────────────────────────────────────────────
PAGE_W     = 842          # landscape-A4 width
PAGE_H     = 1020         # tall page
MARGIN     = 12 * mm
GAP_Y      = 8  * mm
CARD_PAD   = 6
ROW_SHIFT  = 15           # <── nudge every card down 15 pt
TABLE_DOWN = -6

pdf_path = os.path.join(
    out_dir, f"Readiness Report_{name}_{session_date}".replace(":","-") + ".pdf"
)

#  styles
styles = getSampleStyleSheet()
styles.add(ParagraphStyle("cardTitle",
                          fontName="Helvetica-Bold", fontSize=18,
                          textColor=TEXT_COLOR, leading=22))

def draw_card(c, x, y, w, h, title):
    """rounded dark card → returns inner (x,y,w,h) below title"""
    c.setFillColor(CARD_BG); c.setStrokeColor(BRAND_BORDER); c.setLineWidth(2)
    c.roundRect(x, y, w, h, 10, fill=1)
    p = Paragraph(title, styles["cardTitle"])
    _, th = p.wrap(w - 2*CARD_PAD, 40)
    p.drawOn(c, x + CARD_PAD, y + h - th - CARD_PAD)
    return x + CARD_PAD, y + CARD_PAD, w - 2*CARD_PAD, h - th - 2*CARD_PAD

def make_stat_table(rows, avail_w):
    tbl = Table(rows, colWidths=[avail_w*0.45, avail_w*0.30, avail_w*0.25])
    tbl.setStyle(TableStyle([
        # header text
        ("TEXTCOLOR", (0,0), (-1,-1), TEXT_COLOR),
        ("FONTNAME",  (0,0), (-1,0), "Helvetica-Bold"),
        ("FONTSIZE",  (0,0), (-1,0), 7),
    
        # center everything
        ("ALIGN", (0,0), (-1,-1), "CENTER"),
    
        # body rows
        ("TEXTCOLOR", (0,1), (-1,-1), TEXT_COLOR),
        ("FONTNAME",  (0,1), (-1,-1), "Helvetica"),
        ("FONTSIZE",  (0,1), (-1,-1), 7),
    
        # compact vertical padding
        ("VALIGN"       , (0,0), (-1,-1), "BOTTOM"),   # bottom-align text
        ("TOPPADDING",    (0,0), (-1,-1), 0),
        ("BOTTOMPADDING", (0,0), (-1,-1), -2),
    
        # uniform dark background (no blue header bar)
        ("BACKGROUND", (0,0), (-1,-1), CARD_BG),
    
        # thin grid
        ("INNERGRID", (0,0), (-1,-1), 0.2, colors.HexColor("#404040")),
    ]))
    return tbl

# ── BUILD PDF ────────────────────────────────────────────────────────────────
c = canvas.Canvas(pdf_path, pagesize=(PAGE_W, PAGE_H))
c.setFillColor(colors.black); c.rect(0,0,PAGE_W,PAGE_H,stroke=0,fill=1)

# header
c.setFont("Helvetica-BoldOblique", 24); c.setFillColor(TEXT_COLOR)
c.drawString(MARGIN, PAGE_H - MARGIN - 10, "Readiness Screen Report")
c.setFont("Helvetica", 14)
c.drawString(MARGIN, PAGE_H - MARGIN - 30, f"Athlete: {name}")
c.drawString(MARGIN, PAGE_H - MARGIN - 50, f"Date: {session_date}")
c.setStrokeColor(BRAND_BORDER); c.setLineWidth(2)
c.line(MARGIN, PAGE_H - MARGIN - 60, PAGE_W - MARGIN, PAGE_H - MARGIN - 60)

if os.path.exists(LOGO_PATH):
    c.drawImage(
        LOGO_PATH,
        PAGE_W - 260,
        PAGE_H - 95,
        width=210,
        height=84.4,
        preserveAspectRatio=True,
        mask='auto'
    )


# ----------  constants used in every card ----------
GAP_X      = 8                 # horizontal gap between images
GAP_TABLE  = -2                 # gap between table and card-bottom

# ----------  ROW 1 : CMJ  ----------------------------------
CARD_H1 = 105 * mm
cx = MARGIN
cy = PAGE_H - MARGIN - 60 - CARD_H1 - ROW_SHIFT
cw = PAGE_W - 2*MARGIN

ix, iy, iw, ih = draw_card(c, cx, cy, cw, CARD_H1,
                           "Counter-Movement Jump (CMJ)")

# build & wrap the table FIRST  → we now know its height
cmj_rows = [
    ["Metric","Value","%ile"],
    ["Jump Height",  f"{cmj['Jump_Height_CMJ']:.1f}",
                     percentile(cmj_ref["Jump_Height_CMJ"], cmj["Jump_Height_CMJ"])],
    ["PP ForcePlate",f"{cmj['PP_FORCEPLATE_CMJ']:.0f}",
                     percentile(cmj_ref["PP_FORCEPLATE_CMJ"], cmj["PP_FORCEPLATE_CMJ"])],
    ["Force @ PP",   f"{cmj['Force_at_PP_CMJ']:.0f}",
                     percentile(cmj_ref["Force_at_PP_CMJ"], cmj["Force_at_PP_CMJ"])],
    ["Velo @ PP",    f"{cmj['Vel_at_PP_CMJ']:.2f}",
                     percentile(cmj_ref["Vel_at_PP_CMJ"], cmj["Vel_at_PP_CMJ"])],
]
tbl_cmj = make_stat_table(cmj_rows, iw)
tbl_cmj.wrapOn(c, iw, iy - TABLE_DOWN)
tbl_h = tbl_cmj._height            # <─ actual rendered height

# compute graph height now that we know table height
img_w = (iw - 2*GAP_X) / 3         # three images
img_h = ih - tbl_h - GAP_TABLE     # fill remaining space

top_y = iy + ih - img_h            # top edge of graphs

# draw the three graphs
c.drawImage(cmj_hist,    ix,                 top_y,
            img_w, img_h, mask='auto')
c.drawImage(cmj_pp_hist, ix+img_w+GAP_X,     top_y,
            img_w, img_h, mask='auto')
c.drawImage(cmj_scat,    ix+2*(img_w+GAP_X), top_y,
            img_w, img_h, mask='auto')

# finally draw the table flush to bottom
tbl_cmj.drawOn(c, ix, iy + GAP_TABLE)

# ---- ROW 2 : PPU -----------------------------------------------------------
cy = cy - GAP_Y - CARD_H1                    # move down for next card
ix, iy, iw, ih = draw_card(c, cx, cy, cw, CARD_H1,
                           "Plyo Push-Up (PPU)")

# graphs
c.drawImage(ppu_hist,     ix,                 iy+ih-img_h,
            img_w, img_h, mask='auto')
c.drawImage(ppu_pp_hist,  ix+img_w+GAP_X,     iy+ih-img_h,
            img_w, img_h, mask='auto')
c.drawImage(ppu_scat,     ix+2*(img_w+GAP_X), iy+ih-img_h,
            img_w, img_h, mask='auto')


# 4 × 3 stats table
ppu_rows = [
    ["Metric","Value","%ile"],
    ["Jump Height",   f"{ppu['Jump_Height_PPU']:.1f}",
                      percentile(ppu_ref["Jump_Height_PPU"], ppu["Jump_Height_PPU"])],
    ["PP ForcePlate", f"{ppu['PP_FORCEPLATE_PPU']:.0f}",
                      percentile(ppu_ref["PP_FORCEPLATE_PPU"], ppu["PP_FORCEPLATE_PPU"])],
    ["Force @ PP",    f"{ppu['Force_at_PP_PPU']:.0f}",
                      percentile(ppu_ref["Force_at_PP_PPU"], ppu["Force_at_PP_PPU"])],
    ["Velo @ PP",     f"{ppu['Vel_at_PP_PPU']:.2f}",
                      percentile(ppu_ref["Vel_at_PP_PPU"],  ppu["Vel_at_PP_PPU"])],
]

tbl_ppu = make_stat_table(ppu_rows, iw)   # build **once**
tbl_ppu.wrapOn(c, iw, 0)                  # wrap
tbl_ppu.drawOn(c, ix, iy)                 # draw


# ----------------  ROW 3 : Avg-Force hists  ---------------------------------
CARD_H3 = 90 * mm
cy = cy - GAP_Y - CARD_H3                   # next card position
ix, iy, iw, ih = draw_card(c, cx, cy, cw, CARD_H3,
                           "Avg-Force Histograms")

img_w3 = (iw - 30) / 4
for i, img in enumerate(force_imgs):          # I, T, Y, IR90
    c.drawImage(img, ix + i*(img_w3+10), iy + ih - ih, img_w3, ih, mask='auto')

# summary table inside the same card
summary_rows = [["Metric","Value","%ile"]] + [
    [lbl, f"{forces[lbl]:.0f}", percentile(ref_forces[lbl], forces[lbl])]
    for lbl in ["I","T","Y","IR90"]
]
tbl_sum = make_stat_table(summary_rows, iw)
tbl_sum.wrapOn(c, iw, 0)

tbl_x = ix                               # left-align
tbl_y = iy                                # flush with bottom padding
tbl_sum.drawOn(c, tbl_x, tbl_y)



# ---------------------------------------------------------------------------
c.showPage(); c.save()
print("✅ PDF saved →", pdf_path)


✅ PDF saved → D:/Readiness Screen 3/Reports\Readiness Report_Ryan Chasse_2025-10-24.pdf


In [5]:
import shutil

# Readiness Screen DB paths
readiness_db_source = r"D:/Readiness Screen 3/Readiness_Screen_Data_v2.db"
readiness_destination = r"G:/My Drive/Data/Readiness Screen Data"

# Copy Readiness Screen DB
try:
    shutil.copy(readiness_db_source, readiness_destination)
    print(f"Copied {readiness_db_source} → {readiness_destination}")
except Exception as e:
    print(f"Failed to copy {readiness_db_source}: {e}")


Copied D:/Readiness Screen 3/Readiness_Screen_Data_v2.db → G:/My Drive/Data/Readiness Screen Data


In [6]:
# # Creates old Dash Report
# 
# import sqlite3
# import pandas as pd
# from dash import Dash, dcc, html, Input, Output
# import plotly.graph_objects as go
# 
# # ---------------------------
# # Step 1: Data Loading
# # ---------------------------
# 
# conn = sqlite3.connect('D:/Readiness Screen 3/Readiness_Screen_Data_v2.db')
# 
# df_cmj = pd.read_sql_query("SELECT Name, Creation_Date, Jump_Height AS Jump_Height_CMJ, PP_FORCEPLATE AS PP_FORCEPLATE_CMJ, Force_at_PP AS Force_at_PP_CMJ, Vel_at_PP AS Vel_at_PP_CMJ FROM CMJ;", conn)
# df_ppu = pd.read_sql_query("SELECT Name, Creation_Date, Jump_Height AS Jump_Height_PPU , PP_FORCEPLATE AS PP_FORCEPLATE_PPU, Force_at_PP AS Force_at_PP_PPU, Vel_at_PP AS Vel_at_PP_PPU FROM PPU;", conn)
# df_i = pd.read_sql_query("SELECT Name, Creation_Date, Avg_Force AS Avg_Force_I FROM I;", conn)
# df_y = pd.read_sql_query("SELECT Name, Creation_Date, Avg_Force AS Avg_Force_Y FROM Y;", conn)
# df_t = pd.read_sql_query("SELECT Name, Creation_Date, Avg_Force AS Avg_Force_T FROM T;", conn)
# df_ir90 = pd.read_sql_query("SELECT Name, Creation_Date, Avg_Force AS Avg_Force_IR90 FROM IR90;", conn)
# conn.close()
# 
# df_merged = df_cmj.merge(df_ppu, on=["Name", "Creation_Date"], how="outer")
# df_merged = df_merged.merge(df_i, on=["Name", "Creation_Date"], how="outer")
# df_merged = df_merged.merge(df_y, on=["Name", "Creation_Date"], how="outer")
# df_merged = df_merged.merge(df_t, on=["Name", "Creation_Date"], how="outer")
# df_merged = df_merged.merge(df_ir90, on=["Name", "Creation_Date"], how="outer")
# 
# df_merged['Creation_Date'] = pd.to_datetime(df_merged['Creation_Date'])
# 
# df_merged = df_merged.sort_values(by="Creation_Date")
# 
# participants = df_merged['Name'].dropna().unique()
# 
# # full CMJ dataframe for F-v-V scatter (entire db, not just selected user) ### NEW ▶
# cmj_ref = df_cmj.dropna(subset=['Force_at_PP_CMJ', 'Vel_at_PP_CMJ'])
# ppu_ref = df_ppu.dropna(subset=['Force_at_PP_PPU', 'Vel_at_PP_PPU'])
# # ---------------------------
# # Step 2: Initialize the Dash App
# # ---------------------------
# 
# app = Dash(__name__)
# 
# # ---------------------------
# # Step 3: Dash Layout
# # ---------------------------
# 
# app = Dash(__name__, title="Readiness Dashboard")
# 
# app.layout = html.Div([
#     html.H1("Participant Measurements Over Time"),
# 
#     html.Div([
#         html.Label("Select a Participant:"),
#         dcc.Dropdown(
#             id='participant-dropdown',
#             options=[{'label': p, 'value': p} for p in participants],
#             value=participants[0] if len(participants) else None,
#             clearable=False)
#     ], style={'width': '30%', 'display': 'inline-block'}),
# 
#     dcc.Graph(id='measurements-graph'),
#     dcc.Graph(id='jump-heights-graph'),
#     dcc.Graph(id='force-velocity-scatter')          ### NEW ▶
# ])
# 
# # ---------------------------
# # Step 4: Callbacks
# # ---------------------------
# 
# @app.callback(
#     Output('measurements-graph', 'figure'),
#     Output('jump-heights-graph', 'figure'),
#     Output('force-velocity-scatter', 'figure'),
#     Input('participant-dropdown', 'value')
# )
# def update_graph(selected_participant):
#     dff = df_merged[df_merged['Name'] == selected_participant]
# 
#     # First plot: Measurements over time
#     fig_measurements = go.Figure()
#     
#     if 'Avg_Force_I' in dff.columns and dff['Avg_Force_I'].notnull().any():
#         fig_measurements.add_trace(go.Scatter(
#             x=dff['Creation_Date'], y=dff['Avg_Force_I'], mode='lines+markers', name='Avg_Force_I'
#         ))
# 
#     if 'Avg_Force_Y' in dff.columns and dff['Avg_Force_Y'].notnull().any():
#         fig_measurements.add_trace(go.Scatter(
#             x=dff['Creation_Date'], y=dff['Avg_Force_Y'], mode='lines+markers', name='Avg_Force_Y'
#         ))
# 
#     if 'Avg_Force_T' in dff.columns and dff['Avg_Force_T'].notnull().any():
#         fig_measurements.add_trace(go.Scatter(
#             x=dff['Creation_Date'], y=dff['Avg_Force_T'], mode='lines+markers', name='Avg_Force_T'
#         ))
# 
#     if 'Avg_Force_IR90' in dff.columns and dff['Avg_Force_IR90'].notnull().any():
#         fig_measurements.add_trace(go.Scatter(
#             x=dff['Creation_Date'], y=dff['Avg_Force_IR90'], mode='lines+markers', name='Avg_Force_IR90'
#         ))
# 
#     fig_measurements.update_layout(
#         title=f"Measurements Over Time for {selected_participant}",
#         xaxis_title="Date",
#         yaxis_title="Measurement Value",
#         hovermode='x unified'
#     )
# 
#     # Second plot: Jump Heights (CMJ and PPU)
#     fig_jump_heights = go.Figure()
# 
#     if 'Jump_Height_CMJ' in dff.columns and dff['Jump_Height_CMJ'].notnull().any():
#         fig_jump_heights.add_trace(go.Scatter(
#             x=dff['Creation_Date'], y=dff['Jump_Height_CMJ'], mode='lines+markers', name='Jump_Height_CMJ'
#         ))
# 
#     if 'Jump_Height_PPU' in dff.columns and dff['Jump_Height_PPU'].notnull().any():
#         fig_jump_heights.add_trace(go.Scatter(
#             x=dff['Creation_Date'], y=dff['Jump_Height_PPU'], mode='lines+markers', name='Jump_Height_PPU'
#         ))
# 
#     fig_jump_heights.update_layout(
#         title=f"Jump Heights Over Time for {selected_participant}",
#         xaxis_title="Date",
#         yaxis_title="Jump Height (cm)",
#         hovermode='x unified'
#     )
# 
#     # ---------------- force–velocity scatter  -------------------------------
#     # reference cloud – every datapoint in DB
#     fig_scatter = go.Figure()
# 
#     fig_scatter.add_trace(go.Scatter(
#         x=cmj_ref['Vel_at_PP_CMJ'], 
#         y=cmj_ref['Force_at_PP_CMJ'],
#         mode='markers', name='Reference',
#         marker=dict(color='cornflowerblue', opacity=0.45, size=8),
#         hovertemplate="Vel@PP: %{x:.1f}<br>Force@PP: %{y:.0f}<extra></extra>"
#     ))
# 
#     # selected athlete
#     client_rows = cmj_ref[cmj_ref['Name'] == selected_participant]
#     if not client_rows.empty:
#         fig_scatter.add_trace(go.Scatter(
#             x=client_rows['Vel_at_PP_CMJ'], 
#             y=client_rows['Force_at_PP_CMJ'],
#             mode='markers', name=selected_participant,
#             marker=dict(color='red', size=12, line=dict(width=1, color='black')),
#             hovertemplate="Vel@PP: %{x:.1f}<br>Force@PP: %{y:.0f}<extra></extra>"
#         ))
# 
#     fig_scatter.update_layout(
#         title=f"CMJ Force vs Velocity ({selected_participant} highlighted)",
#         xaxis_title="Velocity at PP (m/s)",
#         yaxis_title="Force at PP (N)",
#         template="plotly_dark",
#         legend=dict(bgcolor="rgba(0,0,0,0)")
#     )
# 
#     return fig_measurements, fig_jump_heights, fig_scatter
# # ---------------------------
# # Step 5: Run the App
# # ---------------------------
# if __name__ == '__main__':
#     app.run_server(debug=True)
#     print("Server is running at http://127.0.0.1:5000/")
